## Narrowing the data based on sequencing type
Previously I sorted data to 4 cohort (check "mri_data_sort_to_cohorts.ipynb"). <br>
Now I want to keep only relevant sequencing types: MPRAGE and FSPGR. <br> 
<br>
In my 4 cohorts you can find 3 different naming conventions. I will provide 3 examples, one from each type: <br>
1) 1018_NACC282203_20170908ni <br>
2) mri129ni<br>
3) NACC497363_128401136192134176253428319601354034337135ni<br> 

The 1) all have MPRAGE sequencing, the 2) all have FSPGR sequencing and some of 3) have FSPGR, some MPRAGE.

### 1) 1018_NACC282203_20170908ni
Directory structure: <br>
<br>
1018_NACC282203_20170908ni 
- 1018_NACC282203_20170908 
   - Mag_Images_17_1312211075219452552017090815014550782025439000
   - mIP_ImagesSW_19_1312211075219452552017090815014550780625438000
   - MPRAGE_GRAPPA2_6_1312211075219452552017090814552268696468275000
   - Pha_Images_18_1312211075219452552017090815014550782825440000
   - SWI_Images_20_1312211075219452552017090815014550784425442000
   - T2FLAIRSPACENEW_7_1312211075219452552017090814555267209169171000